#Optimizing Cash Recycler Machine Location


##Background
Cited from ARCA website “A cash recycler is a complex machine that handles a couple of simple, but important tasks—accepting and dispensing cash. It also stores money securely, keeps an accurate accounting of cash on hand, and automates the cash cycle. Generally, you’ll find them in banks, credit unions and back-office retail cash rooms” (ARCA, n.d.).

CRM (Cash Recycler Machine) provided by local banks in Indonesia puts a convenience for their customers to withdraw and deposit their money wherever they want as long there’s a CRM around. CRM also reduces bank operational cost by reducing the need to refill the cash into the old ATM (Automated Teller Machine). But in order to have low maintenance for refilling or taking out the cash form the machine CRM needs a stable cash inflow and outflow. This will be only achievable when there’s an equal withdrawal and deposit transaction. 

By placing CRM in business commerce / where the business is proliferating a cash recycling is ensured. A business commerce can deposit their daily transaction directly through CRM and the customer can withdraw their money from CRM as well. With this kind of traffic a recycling stability is ensured and requires low refilling maintenance.


##Problem and Interest
Currently in Surabaya, Indonesia CRM isn’t as widespread as the ATM but local banks are planning to replace all of the ATMs with CRMs.To create efficient cash recycling, strategic placement is a mandatory. This project aims to optimize placement for CRMs. 


## import all necessary lib 

In [175]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import getpass as pw #for entering credentials

!pip install utm
import utm

print('Libraries imported.')

Libraries imported.


##foursquare id 

In [176]:
#id = pw.getpass("client ID")
CLIENT_ID = pw.getpass("client ID") # your Foursquare ID
CLIENT_SECRET = pw.getpass("client secret") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

client ID··········
client secret··········
Your credentails:


##city of interest

In [177]:
address = 'Surabaya, ID'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Surabaya, ID are -7.2459717, 112.7378266.


In [178]:
# create map of Surabaya using latitude and longitude values
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)
map_sby

##Data collection
Latitude and longitude data of business, venues and commerce in Surabaya acquired using foursquare API. By utilising explore API in hexagon grid pattern we can maximise API call limitation of 950 regular calls per day. 


##Function to retreive data from foursquare
function to retrive venues given specific latitude and longtidude

In [179]:
def getNearbyVenues(latitudes, longitudes, radius=300):
  #modified coursera function (slightly hehe)
  #get all of the venues details by lat and long
  venues_list=[]
  for  lat, lng in zip( latitudes, longitudes):
    try:
      # create the API request URL
      url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
      # make the GET request
      results = requests.get(url).json()["response"]['groups'][0]['items']
      # return only relevant information for each nearby venue
      venues_list.append([(
          lat, 
          lng, 
          v['venue']['name'], 
          v['venue']['location']['lat'], 
          v['venue']['location']['lng'],
          v['venue']['categories'][0]['name']) for v in results])
    except:
      venues_list.append((np.nan,np.nan,np.nan,np.nan,np.nan,np.nan))

  nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  nearby_venues.columns = ['Hexgrid Latitude', 'Hexgrid Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
      
  return(nearby_venues)

In [180]:
SBY_venues = getNearbyVenues(latitudes=[latitude], longitudes=[longitude])
SBY_venues

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,-7.245972,112.737827,Museum Nasional 10 November,-7.245866,112.737800,History Museum
1,-7.245972,112.737827,Ikan Bakar Khas Sulawesi,-7.245508,112.735810,Food Truck
2,-7.245972,112.737827,Restaurant Hai Nan,-7.247611,112.738539,Chinese Restaurant
3,-7.245972,112.737827,Kapin Shark Fin Restaurant,-7.247536,112.739739,Chinese Restaurant
4,-7.245972,112.737827,The Crown,-7.244230,112.738582,Auditorium
5,-7.245972,112.737827,Kantor Gubernur Jawa Timur,-7.245982,112.740294,Government Building
6,-7.245972,112.737827,Viaduct Tugu Pahlawan,-7.245866,112.737800,Historic Site
7,-7.245972,112.737827,Pasar Dadakan Tugu Pahlawan,-7.244411,112.738911,Flea Market
8,-7.245972,112.737827,Bebek Goreng,-7.247306,112.737707,Food Truck
9,-7.245972,112.737827,Ayam Goreng dan Burung Dara Pahlawan,-7.248211,112.738404,Fried Chicken Joint


In [181]:
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(SBY_venues['Venue Latitude'], SBY_venues['Venue Longitude'], SBY_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sby) 
map_sby

##Hexagon Grid Search 
First 30 x 20 hexagon grid is prepared with each of the hexagon radius is 300 meter away from its centroid. Hexagon grid search is used because it covers the maximum amount of area using a radius based search (circle)


##honeycomb grid generator


In [182]:
def honeycomb(x,y,r):
  #x cartesian cordinates for honeycomb grid search
  #zig zag stuff happened here :v
  x = (2*x-1 + y%2 )* r
  #y catresian cordinates for honeycomb grid search
  y = (y * 3 ** (1/2)) * r
  return(x,y)

def gridIterator(x_start, y_start, x_limit, y_limit, radius):
  #this function create hexagonal grid for defined parameter
  #hexagonal grid maximize area coverage if the search being executed radially
  grid_list = []
  for x_coor in range(x_start, x_limit):
    
    for y_coor in range(y_start, y_limit):

      x, y = honeycomb(x_coor, y_coor, radius)
      grid_list.append([x, y])

      pass
    
    pass

  return(grid_list)

def latlong_utm(lat, long):
  #convert latitude and longitude into x y
  x, y, zone, zone_letter = utm.from_latlon(lat, long)

  return([x, y])

def utm_latlong(x, y,  zone, zone_letter):
  #convert x y back into latitude and longitude
  lat, long = utm.to_latlon(x, y, zone, zone_letter)
  
  return([lat, long])

### define grid array size

In [183]:
#create hex grid
coor = pd.DataFrame(gridIterator(0, 0, 30, 25, 300))
coor.head()

,0,1
0,-300,0.000000
1,0,519.615242
2,-300,1039.230485
3,0,1558.845727
4,-300,2078.460969


###change grid lat long into x y coordinates 

In [184]:
#get sby x y coordinates
sby = latlong_utm(latitude, longitude)


###set grid offset

In [185]:
#ofset the x y coordinates by -15000 Meters
sby[0] = sby[0] - 10000
sby[1] = sby[1] - 11000
sby

[681871.1428887163, 9187692.576356351]

In [186]:
coor[0] = coor[0] + sby[0]
coor[1] = coor[1] + sby[1]
coor.head()

,0,1
0,681571.142889,9.187693e+06
1,681871.142889,9.188212e+06
2,681571.142889,9.188732e+06
3,681871.142889,9.189251e+06
4,681571.142889,9.189771e+06


###convert back into lat long

In [187]:
grid_centroid = utm_latlong(coor[0],coor[1], 49, 'M')
grid_centroid = pd.DataFrame(grid_centroid).T

###test honeycomb layout

In [188]:
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng in zip(grid_centroid[0], grid_centroid[1]):
    folium.Circle(
        [lat, lng],
        radius=300,
        color='blue',
        fill=False,
        ).add_to(map_sby) 
        
map_sby

##venues clustering

In [189]:
#SBY_venues = getNearbyVenues(latitudes=grid_centroid[0], longitudes= grid_centroid[1], radius=350)
#saved the retreived to csv to prevent foursquare call limit
SBY_venues = pd.read_csv("drive/MyDrive/coursera/data.csv", index_col=0)
SBY_venues.sample(10)

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,x,y,labels
3264,-7.284199,112.777816,Gae Koen Tok,-7.286060,112.775955,Clothing Store,696064.796397,9.194243e+06,30
2218,-7.317202,112.748063,Plasa Marina,-7.316277,112.748781,Shopping Mall,693050.825610,9.190912e+06,40
1014,-7.298535,112.715390,Tahutek Restu Ibu,-7.300903,112.717384,Diner,689590.415356,9.192626e+06,6
2690,-7.303055,112.761591,Kampoeng Roti,-7.304448,112.762293,Bakery,694548.113382,9.192215e+06,28
3764,-7.279404,112.802246,Solaria,-7.276693,112.803146,Asian Restaurant,699071.715336,9.195267e+06,17
3778,-7.274695,112.804944,Just Coffee,-7.276119,112.803240,Coffee Shop,699082.355445,9.195330e+06,17
2329,-7.270221,112.747880,Colors,-7.271819,112.747354,Pub,692912.328718,9.195830e+06,39
3448,-7.274781,112.783212,Ron's Laboratory,-7.274748,112.781998,Molecular Gastronomy Restaurant,696737.045115,9.195491e+06,8
45,-7.312863,112.652957,Terang Bulan Bangka,-7.313022,112.655416,Food Truck,682742.634786,9.191311e+06,34
840,-7.284471,112.707186,Ikan Bakar Cianjur,-7.287511,112.706779,Seafood Restaurant,688424.939309,9.194112e+06,9


###clustering

After all of the business and venues were gathered, K means Clustering is used for segmenting all data points into separate clusters using x y coordinates. The problem is foursquare explore API returns latitude and longitude value instead of Universal Transverse Mercator or UTM or as we know it as cartesian coordinates. By applying UTM library conversion can be done easily and ready for clustering as seen on ‘x’ and ‘y’ below. 

In [190]:
SBY_venues['x'],SBY_venues['y'] = latlong_utm(SBY_venues['Venue Latitude'].to_numpy(), SBY_venues['Venue Longitude'].to_numpy())
SBY_venues.sample(10)

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,x,y,labels
718,-7.340871,112.701967,martabak mini africa,-7.341527,112.704787,Sandwich Place,688182.311184,9.188138e+06,24
3202,-7.331180,112.778002,Super Indo,-7.331589,112.775069,Supermarket,695947.031461,9.189207e+06,23
2682,-7.303055,112.761591,ENDdustries.Com,-7.300248,112.761472,Office,694459.276628,9.192680e+06,28
3420,-7.274781,112.783212,Häagen-Dazs,-7.274568,112.782022,Ice Cream Shop,696739.763159,9.195511e+06,8
2194,-7.331286,112.750834,The Juice 99,-7.332078,112.748600,Juice Bar,693024.104597,9.189165e+06,3
3317,-7.345263,112.780775,Cifi's Swimming Course,-7.346655,112.779084,Pool,696383.801512,9.187539e+06,23
2721,-7.298368,112.758856,Capture Photography,-7.300709,112.759408,Art Gallery,694231.055608,9.192630e+06,28
3734,-7.255925,112.799436,Pantai Ria Kenjeran,-7.253041,112.799656,Water Park,698696.645799,9.197884e+06,42
2806,-7.260782,112.758709,Tahu Campur Pak Mat Andri,-7.259192,112.757010,Indonesian Restaurant,693984.137885,9.197222e+06,21
3269,-7.279511,112.775081,Shao Kao,-7.280425,112.777045,BBQ Joint,696187.593911,9.194865e+06,8


###K-means clustering
Clustering is executed by using x and y cartesian coordinates data and segmenting it for n clusters. Then the centroid of the cluster is retrieved to get an optimal location for the CRMs 


In [204]:
from sklearn.cluster import KMeans

####K = 40

In [205]:
kclusters = 40
clustering = KMeans(n_clusters=kclusters).fit(SBY_venues.loc[:,['x','y']])
SBY_venues['labels'] = clustering.labels_
SBY_venues.sample(10)

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,x,y,labels
587,-7.284532,112.690887,Sop Kaki Sapi Pak Pri,-7.283567,112.689142,Indonesian Restaurant,686478.883927,9.194555e+06,34
339,-7.284593,112.674587,Zhang Palace,-7.284767,112.676665,Event Space,685100.571982,9.194427e+06,9
1205,-7.331370,112.729100,Ayam Bakar Primarasa,-7.330065,112.730420,Indonesian Restaurant,691017.505429,9.189395e+06,16
1844,-7.260866,112.736978,Seiko Showroom & Service Center,-7.260706,112.739613,Watch Shop,692062.209427,9.197062e+06,3
3082,-7.284220,112.772383,Atlas Sports Club,-7.286000,112.771046,Gym / Fitness Center,695522.649930,9.194251e+06,33
267,-7.312783,112.674692,Frolino Bakery & Pastry,-7.313578,112.675888,Bakery,685002.915158,9.191241e+06,18
1395,-7.298473,112.731690,Indomaret,-7.296191,112.731189,Convenience Store,691116.854028,9.193141e+06,38
2357,-7.270221,112.747880,D'Cost,-7.270062,112.745393,Seafood Restaurant,692696.558145,9.196025e+06,39
3538,-7.331137,112.788869,Indomaret,-7.331718,112.786789,Convenience Store,697241.144679,9.189188e+06,5
622,-7.261051,112.688082,Mascot Red Ranchu Showroom Fish,-7.259390,112.689168,Farm,686491.738610,9.197229e+06,30


In [206]:
centroid = pd.DataFrame(clustering.cluster_centers_)
centroid = utm_latlong(centroid[0],centroid[1], 49, 'M')
centroid = pd.DataFrame(centroid).T
centroid = centroid.reset_index()

#####visualize
N centroids were chosen as a baseline for analysis and the circle is drawn with radius 1km to visualize 1km coverage. The result is shown below

In [207]:
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
from folium.features import DivIcon

venues_sample = SBY_venues#.sample(2000)
for lat, lng, borough, neighborhood, cluster in zip(venues_sample['Venue Latitude'], venues_sample['Venue Longitude'], venues_sample['Venue Category'], venues_sample['Venue'], venues_sample['labels']):
    #label = '{}, {}'.format(neighborhood, borough)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        #popup=label,
        color=rainbow[cluster-1],
        opacity = .8,
        fill=False,
        parse_html=False).add_to(map_sby)   

for centroid_lat, centroid_long, index in zip(centroid[0],centroid[1], centroid['index']):
  folium.Marker(
    [centroid_lat, centroid_long],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 18pt; color : {}">{}</div>'.format("000000",index),
        )).add_to(map_sby)
  folium.Circle(
    [centroid_lat, centroid_long],
    radius=1000,
    #popup=label,
    color='black',
    opacity = .8,
    fill=False,
    parse_html=False).add_to(map_sby)   


map_sby

Output hidden; open in https://colab.research.google.com to view.

####K = 50

In [208]:
kclusters = 50
clustering = KMeans(n_clusters=kclusters).fit(SBY_venues.loc[:,['x','y']])
SBY_venues['labels'] = clustering.labels_
SBY_venues.sample(10)

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,x,y,labels
211,-7.317511,112.666559,D'stupid baker,-7.316632,112.668701,Bakery,684208.050322,9.190907e+06,15
3699,-7.288822,112.796851,Indomaret,-7.289686,112.798195,Convenience Store,698519.236089,9.193832e+06,35
1096,-7.237447,112.717874,boezem morokrembangan,-7.235260,112.719638,Lake,689866.859893,9.199885e+06,27
735,-7.312681,112.701860,Milk Me,-7.313590,112.699608,Café,687622.132976,9.191230e+06,22
3632,-7.265342,112.794041,Kafetien 88,-7.266123,112.796183,Coffee Shop,698307.345637,9.196439e+06,23
2850,-7.317139,112.764363,Bebek Goreng Sejedewe,-7.318887,112.763057,Restaurant,694626.180267,9.190618e+06,47
829,-7.289179,112.704487,NAV,-7.287400,112.706700,Karaoke Bar,688416.227690,9.194124e+06,48
1649,-7.293743,112.739821,Grand Darmo Suite,-7.292008,112.740758,Hotel,692175.258958,9.193600e+06,10
3784,-7.274695,112.804944,A&W,-7.276745,112.805831,American Restaurant,699368.215920,9.195260e+06,38
1019,-7.293826,112.718088,Ciputra World XXI,-7.292384,112.719356,Multiplex,689811.721726,9.193567e+06,1


In [209]:
centroid = pd.DataFrame(clustering.cluster_centers_)
centroid = utm_latlong(centroid[0],centroid[1], 49, 'M')
centroid = pd.DataFrame(centroid).T
centroid = centroid.reset_index()

#####visualize
N centroids were chosen as a baseline for analysis and the circle is drawn with radius 1km to visualize 1km coverage. The result is shown below

In [210]:
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
from folium.features import DivIcon

venues_sample = SBY_venues#.sample(2000)
for lat, lng, borough, neighborhood, cluster in zip(venues_sample['Venue Latitude'], venues_sample['Venue Longitude'], venues_sample['Venue Category'], venues_sample['Venue'], venues_sample['labels']):
    #label = '{}, {}'.format(neighborhood, borough)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        #popup=label,
        color=rainbow[cluster-1],
        opacity = .8,
        fill=False,
        parse_html=False).add_to(map_sby)   

for centroid_lat, centroid_long, index in zip(centroid[0],centroid[1], centroid['index']):
  folium.Marker(
    [centroid_lat, centroid_long],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 18pt; color : {}">{}</div>'.format("000000",index),
        )).add_to(map_sby)
  folium.Circle(
    [centroid_lat, centroid_long],
    radius=1000,
    #popup=label,
    color='black',
    opacity = .8,
    fill=False,
    parse_html=False).add_to(map_sby)   


map_sby

Output hidden; open in https://colab.research.google.com to view.

####K = 60

In [211]:
kclusters = 60
clustering = KMeans(n_clusters=kclusters).fit(SBY_venues.loc[:,['x','y']])
SBY_venues['labels'] = clustering.labels_
SBY_venues.sample(10)

,Hexgrid Latitude,Hexgrid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,x,y,labels
3561,-7.298261,112.786022,In Music Studio,-7.298960,112.786819,Music Venue,697258.894046,9.192811e+06,30
3028,-7.317117,112.769796,Ruang Pamer Kedaung Group,-7.318698,112.769570,Furniture / Home Store,695345.485196,9.190636e+06,38
984,-7.336121,112.715533,jogging track Masjid Agung Surabaya,-7.336865,112.716379,Track,689464.234188,9.188649e+06,11
1055,-7.289138,112.715354,Ajinoya Japanese Street Food,-7.286351,112.715090,Asian Restaurant,689343.176303,9.194236e+06,28
1957,-7.340704,112.745437,Raya Kutisari Indah,-7.339895,112.744399,Park,692556.884272,9.188302e+06,24
2873,-7.298346,112.764289,Nasi Goreng Mafia,-7.299728,112.765871,Indonesian Restaurant,694945.236082,9.192735e+06,35
18,-7.294089,112.647454,Ciputra Park Surabaya,-7.296534,112.648745,Other Great Outdoors,682012.624879,9.193137e+06,16
1723,-7.279659,112.737050,D'Cafesera,-7.279139,112.738279,Food Court,691907.027508,9.195024e+06,20
268,-7.312783,112.674692,Kolam renang apartement puncak permai tower A,-7.312171,112.676057,Pool,685022.130248,9.191397e+06,45
3137,-7.265428,112.772310,Indomaret Pojok Kaliwaron,-7.263905,112.772331,Market,695674.177357,9.196694e+06,29


In [212]:
centroid = pd.DataFrame(clustering.cluster_centers_)
centroid = utm_latlong(centroid[0],centroid[1], 49, 'M')
centroid = pd.DataFrame(centroid).T
centroid = centroid.reset_index()

#####visualize
N centroids were chosen as a baseline for analysis and the circle is drawn with radius 1km to visualize 1km coverage. The result is shown below

In [213]:
map_sby = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
from folium.features import DivIcon

venues_sample = SBY_venues#.sample(2000)
for lat, lng, borough, neighborhood, cluster in zip(venues_sample['Venue Latitude'], venues_sample['Venue Longitude'], venues_sample['Venue Category'], venues_sample['Venue'], venues_sample['labels']):
    #label = '{}, {}'.format(neighborhood, borough)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        #popup=label,
        color=rainbow[cluster-1],
        opacity = .8,
        fill=False,
        parse_html=False).add_to(map_sby)   

for centroid_lat, centroid_long, index in zip(centroid[0],centroid[1], centroid['index']):
  folium.Marker(
    [centroid_lat, centroid_long],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 18pt; color : {}">{}</div>'.format("000000",index),
        )).add_to(map_sby)
  folium.Circle(
    [centroid_lat, centroid_long],
    radius=1000,
    #popup=label,
    color='black',
    opacity = .8,
    fill=False,
    parse_html=False).add_to(map_sby)   


map_sby

Output hidden; open in https://colab.research.google.com to view.

In [214]:
SBY_venues.to_csv("drive/MyDrive/coursera/data.csv")

##Discussion
With 50 CRMs deployed around the centroid of the cluster, we can serve most of the business and venues while maintaining a low number of CRMs. Almost all of the CRMs are within reach of 1km inside the clusters or 10 - 15 minutes walk from the furthest point of the cluster. 
##Conclusion
Deployment of 50 CRMs within the centroid of the cluster can maximize coverage within 1km radius. Low number of CRMs greatly reduce operational cost whilst maintaining maximum coverage.
##Reference
ARCA. (n.d.). Cash Recycling. ARCA. https://arca.com/solutions/cash-recycling#:~:text=A%20cash%20recycler%20is%20a,back%2Doffice%20retail%20cash%20rooms.
